In [8]:
#r "nuget: Fli"

open Fli
open System.IO

let postsFlaDir = "/Users/flavioc/Web/fcolavecchia/_fsharp"
let srcDir = "/Users/flavioc/Documents/Blog/"



Installed Packages Fli, 1.0.1

In [3]:
let nbConvert file outfile =
    let fullName = Path.Combine(srcDir,"en",file)
    $"/opt/miniconda3/bin/jupyter nbconvert --to markdown {fullName}.ipynb --no-prompt --output={outfile}"

In [4]:
let cliConvert inFile outfile = 
    cli {
        Shell BASH 
        Command ("\"" + (nbConvert inFile outfile) + "\"" )
        // Arguments $"--output={outfile}" 
        WorkingDirectory (Path.Combine(srcDir,"tools"))
    }

let dry (cliValue:  ShellContext) =    
    cliValue 
    |> Command.toString

let run (cliValue:  ShellContext) =    
    cliValue 
    |> Command.execute


In [13]:
let allNotebooks = Directory.GetFiles(Path.Combine(srcDir,"en/"),"*.ipynb")
let notebooks = 
    [
        "Fundamentals"
        "Functions"
        "IntroToTypes"
        "DiscriminatedUnions"
        "Records"
        "TypeExercises"
    ]
    
notebooks

index,value
0,Fundamentals
1,Functions
2,IntroToTypes
3,DiscriminatedUnions
4,Records
5,TypeExercises


In [14]:
let posts = 
    notebooks
    // |> List.mapi (fun i n -> string i + "_"+n)
    |> List.map (fun n -> Path.Combine(postsFlaDir,n+".md"))
    // |> List.map (fun (ni,no) -> dry (cliConvert ni no))
posts

index,value
0,/Users/flavioc/Web/fcolavecchia/_fsharp/Fundamentals.md
1,/Users/flavioc/Web/fcolavecchia/_fsharp/Functions.md
2,/Users/flavioc/Web/fcolavecchia/_fsharp/IntroToTypes.md
3,/Users/flavioc/Web/fcolavecchia/_fsharp/DiscriminatedUnions.md
4,/Users/flavioc/Web/fcolavecchia/_fsharp/Records.md
5,/Users/flavioc/Web/fcolavecchia/_fsharp/TypeExercises.md


In [17]:
(notebooks, posts)
    ||> List.map2 (fun ni no -> run (cliConvert ni no))

index Id Text ExitCode Error 0 91938 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/Fundamentals.ipynb to markdown
[NbConvertApp] Writing 8666 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/Fundamentals.md 1 91941 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/Functions.ipynb to markdown
[NbConvertApp] Writing 12049 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/Functions.md 2 91949 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/IntroToTypes.ipynb to markdown
[NbConvertApp] Writing 13524 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/IntroToTypes.md 3 91954 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/DiscriminatedUnions.ipynb to markdown
[NbConvertApp] Writing 9648 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/DiscriminatedUnions.md 4 91961 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/Records.ipynb to markdown
[NbConvertApp] Writing 4894 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/Records.md 5 91969 <null> 0 Value [NbConvertApp] Converting notebook /Users/flavioc/Documents/Blog/en/TypeExercises.ipynb to markdown
[NbConvertApp] Writing 1980 bytes to /Users/flavioc/Web/fcolavecchia/_fsharp/TypeExercises.md

In [ ]:
(notebooks, posts)
    ||> List.map2 (fun ni no -> run (cliConvert ni no))

In [ ]:
let test =
    cli {
        Shell BASH
        Command "echo Hello World!"
    }
    // |> Command.execute
test

In [20]:
let frontmatter title tagline = 
    $"---
layout: post
title:  {title}
tagline: {tagline}
categories: 
- F# as your first functional programming language
tags:
- fsharp
---
"

In [ ]:
printfn "%A" (frontmatter "hola" " To completely define a function, you not only need to establish its behavior, but also its inputs and outputs. That is what _types_ are for! )

In [21]:
let sent = @"These guides are oriented to those progammers interested in learning some concepts on F\# functional programming, from a practical perspective. F\# is an excelent _first_ functional programming language: it is functional (of course), it has a clean and readable syntax (not a lot of fancy symbols and stuff), it is flexible (in case you need to grasp some other paradigm in the middle of your code) and it is concise enough to express your ideas with clarity. "

let findFirstSentence s =
    let regex = @"^(.*?[\.!\?](?:\s|$))"
    let rx = System.Text.RegularExpressions.Regex(regex, System.Text.RegularExpressions.RegexOptions.Compiled)

    let check e =
        let m = rx.Match(e)
        m.Value//, m.Groups[1], m.Groups[2])

    check s 

printfn "%A" (findFirstSentence sent)
        



"These guides are oriented to those progammers interested in learning some concepts on F\# functional programming, from a practical perspective. "


In [26]:
let createFrontMatter post =
    let lines = System.IO.File.ReadLines(post)
                |> Seq.toList

    let title = 
        let firstline = lines.[0]
        firstline.Replace("## ","")

    let firstline = 
        lines[1..10]
        |> List.tryFindIndex (fun l -> l.Contains("##"))   
        |> Option.defaultValue 1     


    let tagline = lines[1..10]  
                |> List.tryFind (fun s -> 
                                    String.IsNullOrEmpty(s) |> not &&
                                    s.Contains("##") |> not)   
                |> Option.map findFirstSentence 
                |> Option.defaultValue ""    

    // printfn "%A" title
    // printfn "%A" tagline
    let front = frontmatter title tagline 
    let newpost = [front] @ lines[firstline..]
    let newFile = post.Replace(".md","_new.md")
    // printfn "%A" newFile
    File.AppendAllLines(newFile, newpost)
    
    // printfn "%A" (newpost[0..20] |> String.Concat)
    // newpost

// printfn "%A" posts
// createFrontMatter posts.[0]    

In [27]:
posts 
|> List.map createFrontMatter 


index,value
0,<null>
1,<null>
2,<null>
3,<null>
4,<null>
5,<null>
